In [2]:
import pandas as pd
import numpy as np

C:\Users\wuron\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [27]:
import json

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [ ]:
lst_sl = pd.read_csv('listings_salem.csv')
nbh_sl = pd.read_csv('neighbourhoods_salem.csv')
rv_sl = pd.read_csv('reviews_salem.csv')
cld_sl = pd.read_csv('calendar_salem.csv')

In [ ]:
nrows_listing_la = lst_la.shape[0]
nrows_listing_pl = lst_pl.shape[0]
nrows_listing_sl = lst_sl.shape[0]
nrows_listing_sd = lst_sd.shape[0]
dropColListing = []
for col in(lst_la.columns):
    temp = [100 * lst_la[col].isna().sum() / nrows_listing_la > 40, 
            100 * lst_pl[col].isna().sum() / nrows_listing_pl > 40, 
            100 * lst_sl[col].isna().sum() / nrows_listing_sl > 40, 
            100 * lst_sd[col].isna().sum() / nrows_listing_sd > 40]
    if (any(temp)):
        dropColListing.append(col)
        
# print(dropColListing)
lst_la = lst_la.drop(columns=dropColListing)

lst_pl = lst_pl.drop(columns=dropColListing)

lst_sl = lst_sl.drop(columns=dropColListing)

lst_sd = lst_sd.drop(columns=dropColListing)
# ['neighborhood_overview', 'host_about', 'neighbourhood', 'neighbourhood_group_cleansed', 
# 'bathrooms', 'calendar_updated', 'license']

In [ ]:
lst_la.to_csv('listings_la_cleaned.csv', index=False)
lst_pl.to_csv('listings_pl_cleaned.csv', index=False)
lst_sl.to_csv('listings_sl_cleaned.csv', index=False)
lst_sd.to_csv('listings_sd_cleaned.csv', index=False)

In [31]:
# change the csv file and the output filename for different cities
lst = pd.read_csv('listings_sd_cleaned.csv')
# lst_sl = lst_sl.drop(columns=['Unnamed: 0'])
dic_lst = {}
types = {}
for i in range(len(lst.columns)):
#     print(lst.columns[i], lst.dtypes[i]=='float64')
    currcol = lst.columns[i]
    if currcol in ['price', 'host_response_rate', 'host_acceptance_rate']:
        types[currcol] = float
        continue
    currtype = lst.dtypes[i]
    newtype = ''
    if currtype=='int64':
        newtype = int
    elif currtype==object:
        newtype = str
    elif currtype=='float64':
        newtype = float
    types[currcol] = newtype



In [32]:
final = {}
final_arr = []
for j in range(lst.shape[0]):
    temp = {}
    for col in lst.columns:
        currcontent = lst.iloc[j][col]
        if col=='price':
            temp[col] = float(currcontent.replace('$','').replace(',',''))
        elif col=='host_response_rate':
            if pd.isnull(currcontent):
                temp[col] = -1
            else:
                t = str(currcontent).replace('%','')
                temp[col] = float(t) / 100.0
        elif col=='host_acceptance_rate':
            if pd.isnull(currcontent):
                temp[col] = -1
            else:
                t = str(currcontent).replace('%','')
                temp[col] = float(t) / 100.0
        else:
            temp[col] = types[col](currcontent)
    final_arr.append(temp)
final["all_listings"] = final_arr
    
with open("listing_<cityname>.json", "w") as file:
    json.dump(final, file, indent=4, sort_keys=True,
              separators=(', ', ': '),
              cls=NpEncoder)